In [3]:
import sys
sys.path.append('..')
import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
import torchvision
import random
import os
import cv2
from matplotlib import pyplot as plt
from detr import detr_pro_dataset as ds
from common import image
from common.config import val_annotation_file, val_img_od_dict_file, val_img_dir, \
    train_annotation_file, train_img_od_dict_file, train_img_dir, img_size, max_img_len
from detr.config import cid_to_name
from torchvision import transforms
from detr import detr_pro, examine, anno, anchor, detr_model_base
import random
%matplotlib inline

In [4]:
dicts_val = anno.build_img_dict(val_annotation_file, val_img_od_dict_file, task='od')
dicts_train = anno.build_img_dict(train_annotation_file, train_img_od_dict_file, task='od')

In [5]:
device = torch.device("mps")

model = detr_pro.DETR(dmodel=320, dhead=64,n_enc_layer=20, n_dec_layer=8, exam_diff=True)
model.to(device)
n_head = 5

In [199]:
saved_state = torch.load(f'/Users/zx/Documents/ml/restart/resources/od_pro_1130.pt',  map_location=device)
model.load_state_dict(saved_state)
model.train = False
extractor = examine.FeatureExtractor(model, layers=[m[0] for m in model.named_modules()])

In [ ]:
data = ds.DetrProDS(val_img_dir,dicts_val, sample_num=10, random_flip=True)
dl = DataLoader(data, batch_size=1, shuffle=True)

for img, boxes_gt_xyxy, cids, img_id in dl:
    img = img.to(device)
    img, anchors_, anchors_new, boxes, names, attns1, attns2, n_head, boxes_pred_xyxy, cls_pred = examine.examine_attn(
            img, extractor, n_head, device, 
            'decoder.ca_layers.0.self_attn.q_proj', 'decoder.ca_layers.0.self_attn.k_proj', 
            'decoder.ca_layers.5.self_attn.q_proj', 'decoder.ca_layers.5.self_attn.k_proj')

    examine.draw_attn(img, anchors_, anchors_new, boxes, names, attns1, n_head)
    # examine.draw_attn(img, anchors_, anchors_new, boxes, names, attns2, n_head)
    break


In [ ]:
bsz = 1
sample_num=10
fig, axes = plt.subplots(1, sample_num, figsize=(sample_num * 4, 4))
data = ds.DetrProDS(val_img_dir,dicts_val, sample_num=10, random_flip='horizontal')
dl = DataLoader(data, batch_size=bsz, shuffle=True)
i = 0
for img, boxes_gt_xyxy, cids, img_id in dl:
    print(img_id)
    img = torch.permute(img, [0, 2, 3, 1])[0]
    axes[i].axis('off')
    axes[i].imshow(img)
#     axes[i].legend('ooo')
    for box in boxes_gt_xyxy[0]:
        x1, y1, x2, y2 = (box * max_img_len).tolist()
        axes[i].add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', lw=1))
    i += 1

In [217]:
import cv2
import time
import numpy as np
 
def video_demo(cids=None):
    capture = cv2.VideoCapture(0)#0为电脑内置摄像头
    fps = 0.0
    while(True):
        start_time = time.time()
        ret, frame = capture.read()#摄像头读取,ret为是否成功打开摄像头,true,false。 frame为视频的每一帧图像
        frame = cv2.flip(frame, 1)#摄像头是和人对立的，将图像左右调换回来正常显示。
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = torch.tensor(frame / 256)
        # print(frame.shape)
        
        # img, offset_h, offset_w = image.pad_img(frame, random_offset=False, content='zero', out_ratio=img_size[0] / img_size[1])
        # resize_factor = img_size[0] / img.shape[0]
        img = torch.permute(frame, [2, 0, 1])
        img = torchvision.transforms.Resize((288, 512))(img)
        img = img.to(torch.float32)

        img_ = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
        img_ = torch.unsqueeze(img_, 0)
        img_ = img_.to(device)

        with torch.no_grad():
            boxes_pred_xyxy, cls_logits_pred, _, _, _, _, src_cids, _, _ = model(img_, cids_gt_batch=cids)
            # boxes_pred_xyxy, cls_logits_pred, _, _, _, _, _, _, _ = model(img_, cids_gt_batch=[[1]])
        # print(cls_logits_pred.shape)
        img = torch.permute(img, [1, 2, 0]).contiguous()
        img = img.cpu().numpy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        if boxes_pred_xyxy is not None:
            boxes_pred_xyxy = (boxes_pred_xyxy * max_img_len).to(int)
            cls_pred = cls_logits_pred.argmax(-1)
            
            n_query = cls_pred.shape[1]
            cnt = 0
            for i in range(n_query):
                cls = cls_pred[0][i].item()
                if cls == 0:
                    continue
                cnt += 1
                name = cid_to_name.get(cls)
                x1, y1, x2, y2 = boxes_pred_xyxy[0, i]
                x1, y1, x2, y2 = x1.item(), y1.item(), x2.item(), y2.item()
                rgb = (random.randint(0, 255) / 255,random.randint(0, 255) / 256,random.randint(0, 255) / 256)
                
                cv2.rectangle(img, (x1, y1), (x2, y2), rgb, 2)
                
                cv2.putText(img, name, (x1, y1 + 8), cv2.FONT_HERSHEY_SIMPLEX,  0.5, rgb , 2)
            print(f'{cnt} objects!', [cid_to_name[c] for c in src_cids[0]])
        else:
            print(f'{0} objects!')
        
        end_time = time.time()
        fps = ( fps + (1. / (end_time - start_time)) ) / 2
        cv2.putText(img, f"fps={fps:.2f}", (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.imshow("video", img)
        c = cv2.waitKey(50)
        if c == 27:
            cv2.destroyAllWindows()
            break


In [218]:
# video_demo(cids=[[1]])
video_demo()

17 objects! ['person', 'horse']
15 objects! ['tv', 'person']
16 objects! ['tv', 'person', 'couch']
5 objects! ['person']
15 objects! ['tv', 'person']
5 objects! ['person']
5 objects! ['person']
5 objects! ['person']
5 objects! ['person']
12 objects! ['person', 'couch']
15 objects! ['tv', 'person']
13 objects! ['person', 'couch']
12 objects! ['person', 'couch']
12 objects! ['person', 'couch']
5 objects! ['person']
4 objects! ['person']
4 objects! ['person']
15 objects! ['person', 'dining table', 'oven']
4 objects! ['person']
15 objects! ['person', 'dining table', 'oven']
12 objects! ['person', 'oven']
15 objects! ['person', 'dining table', 'oven']
4 objects! ['person']
10 objects! ['person', 'dining table']
4 objects! ['person']
10 objects! ['person', 'dining table']
4 objects! ['person']
10 objects! ['person', 'dining table']
4 objects! ['person']
4 objects! ['person']
14 objects! ['person', 'dining table', 'oven']
4 objects! ['person']
12 objects! ['person', 'oven']
4 objects! ['perso

In [52]:
# device

In [53]:
import random

In [28]:
random.sample([4,5,6,7], k=0)

[]

In [219]:
saved_state

KeyboardInterrupt: 